## Test

In [46]:
def test(func: 'knapsack solver', out=True):
    P1, W1, C1 = [1, 6, 10, 16], [1, 2, 3, 5] , 7
    if out: 
        print(f'Profits: {P1}, weights: {W1}, capacity: {C1}, answer: {func(P1, W1, C1)}')
    P2, W2, C2 = [1, 6, 10, 16], [1, 2, 3, 5] , 6
    if out:
        print(f'Profits: {P2}, weights: {W2}, capacity: {C2}, answer: {func(P2, W2, C2)}')
    P3, W3, C3 = [1, 6, 10, 16], [1, 2, 3, 5] , 5
    if out:
        print(f'Profits: {P3}, weights: {W3}, capacity: {C3}, answer: {func(P3, W3, C3)}')

## Brute force

In [47]:
def solve_knapsack_brute(profits: "List[int]", weights: "List[int]", capacity: int) -> int:
    return helper_brute(profits, weights, capacity, 0)

def helper_brute(profits, weights, capacity, curIdx) -> int:
    if capacity<=0 or curIdx >= len(profits):
        return 0
    
    profitWithCur = 0
    if weights[curIdx] <= capacity:
        profitWithCur = profits[curIdx] + helper(profits, weights, capacity - weights[curIdx], curIdx+1)
        
    profitWoutCur = helper(profits, weights, capacity, curIdx+1)
    
    return max(profitWithCur, profitWoutCur)

In [48]:
test(solve_knapsack_brute)

Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 7, answer: 22
Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 6, answer: 17
Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 5, answer: 16


In [49]:
%timeit test(solve_knapsack_brute, out=False)

406 ns ± 5.93 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Top-down DP

In [50]:
def solve_knapsack_DP_topdown(profits: "List[int]", weights: "List[int]", capacity: int) -> int:
    dp = [[-1 for x in range(capacity+1)] for y in range(len(profits))]
    return helper_DP_topdown(dp, profits, weights, capacity, 0)

def helper_DP_topdown(dp, profits, weights, capacity, curIdx) -> int:
    if capacity<=0 or curIdx >= len(profits):
        return 0
    
    if dp[curIdx][capacity] == -1:
    
        profitWithCur = 0
        if weights[curIdx] <= capacity:
            profitWithCur = profits[curIdx] + helper(profits, weights, capacity - weights[curIdx], curIdx+1)
        
        profitWoutCur = helper(profits, weights, capacity, curIdx+1)
    
        dp[curIdx][capacity] = max(profitWithCur, profitWoutCur)
        
    return dp[curIdx][capacity]

In [51]:
test(solve_knapsack_DP_topdown)

Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 7, answer: 22
Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 6, answer: 17
Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 5, answer: 16


In [52]:
%timeit test(solve_knapsack_DP_topdown, out=False)

383 ns ± 3.54 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Bottom-up DP

In [53]:
def solve_knapsack_DP_bottomup(profits: "List[int]", weights: "List[int]", capacity: int) -> int:
    dp = [[0 for x in range(capacity+1)] for y in range(len(profits))]
    
    for c in range(capacity+1):
        if weights[0] <= c:
            dp[0][c] = profits[0]
            
            
    for i in range(1, len(profits)):
        for c in range(1, capacity+1):
            profitWithCur, profitWoutCur = 0, 0
            if weights[i] <= c:
                profitWithCur = profits[i] + dp[i-1][c-weights[i]]
            profitWoutCur = dp[i-1][c]
            #print(f'object={i}, capacity={c}, inc={profitWithCur}, notinc={profitWoutCur}')
            dp[i][c] = max(profitWithCur, profitWoutCur)
     
    print_selected_elements(dp, weights, profits, capacity)
    return dp[-1][-1]

def print_selected_elements(dp, weights, profits, capacity):
    print("Selected weights are: ", end="")
    n = len(weights)
    totalProfit = dp[n-1][capacity]
    for i in range(n-1, 0, -1):
        if totalProfit != dp[i-1][capacity]:
            print(str(weights[i]) + " ", end="")
            capacity -= weights[i]
            totalProfit -= profits[i]
    if totalProfit != 0:
        print(str(weights[0])+ " ", end='')
    print()

In [54]:
test(solve_knapsack_DP_bottomup)

Selected weights are: 5 2 
Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 7, answer: 22
Selected weights are: 3 2 1 
Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 6, answer: 17
Selected weights are: 3 2 
Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 5, answer: 16


## DP bottom-up [optimized]

In [57]:
def solve_knapsack_DP_bottomup_optimized(profits: "List[int]", weights: "List[int]", capacity: int) -> int:
    dp = [[0 for _ in range(capacity+1)] for y in range(2)]
    
    for c in range(capacity+1):
        if weights[0] <= c:
            dp[0][c] = profits[0]
            
            
    for i in range(1, len(profits)):
        for c in range(1, capacity+1):
            profitWithCur, profitWoutCur = 0, 0
            if weights[i] <= c:
                profitWithCur = profits[i] + dp[(i-1)%2][c-weights[i]]
            profitWoutCur = dp[(i-1)%2][c]
            #print(f'object={i}, capacity={c}, inc={profitWithCur}, notinc={profitWoutCur}')
            dp[i%2][c] = max(profitWithCur, profitWoutCur)
     
    return dp[-1][-1]

In [58]:
test(solve_knapsack_DP_bottomup_optimized)

Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 7, answer: 22
Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 6, answer: 17
Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 5, answer: 16


In [69]:
def solve_knapsack_DP_bottomup_optimized_more(profits: "List[int]", weights: "List[int]", capacity: int) -> int:
    dp = [0 for _ in range(capacity+1)]
    
    for c in range(capacity+1):
        if weights[0] <= c:
            dp[c] = profits[0]
            
            
    for i in range(1, len(profits)):
        for c in range(capacity, 0, -1):
            profitWithCur, profitWoutCur = 0, 0
            if weights[i] <= c:
                profitWithCur = profits[i] + dp[c-weights[i]]
            profitWoutCur = dp[c]
            #print(f'object={i}, capacity={c}, inc={profitWithCur}, notinc={profitWoutCur}')
            dp[c] = max(profitWithCur, profitWoutCur)
     
    return dp[-1]

In [70]:
test(solve_knapsack_DP_bottomup_optimized_more)

Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 7, answer: 22
Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 6, answer: 17
Profits: [1, 6, 10, 16], weights: [1, 2, 3, 5], capacity: 5, answer: 16
